<a href="https://colab.research.google.com/github/preethamgoud9/fashion-mnist-on-gpu/blob/main/ann_fashion_mnist_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import matplotlib.pyplot as plt

# New Section

In [ ]:
torch.manual_seed(42)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"using devide:{device}")

using devide:cuda


In [ ]:
df = pd.read_csv("fashion-mnist_train.csv")
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#train test split
x = df.iloc[:,1:].values
y = df.iloc[:,0].values

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y , test_size=0.2 , random_state=42)

In [ ]:
#scaling the features, basically we are normalizing the values to an smaller intervals (0-1)
x_train = x_train/255.0
x_test = x_test/255.0

In [ ]:
#creating custom dataset
class CustomDataset(Dataset):

    def __init__(self,features,labels):

        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels,dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self,index):
        return self.features[index],self.labels[index]

In [ ]:
#create train_dataset object
train_dataset = CustomDataset(x_train,y_train)

In [ ]:
len(train_dataset)

48000

In [ ]:
#create train_dataset object
test_dataset = CustomDataset(x_test,y_test)

In [ ]:
len(test_dataset)

12000

In [ ]:
#creating train and test loader
train_loader = DataLoader(train_dataset,batch_size=32,shuffle=True)
test_loader = DataLoader(test_dataset,batch_size=32,shuffle=False)

In [ ]:
#define nn class
class MyNN(nn.Module):

    def __init__(self,num_features):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(num_features,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,10)
        )
    def forward(self,x):
        return self.model(x)

In [ ]:
#set learning rate and epochs
learning_rate = 0.1
epochs = 100

In [ ]:
#instantiate the model
model = MyNN(x_train.shape[1])
model = model.to(device)

#loss function
criterion = nn.CrossEntropyLoss()

#optimizer
optimizer = optim.SGD(model.parameters(),lr =learning_rate)

In [ ]:
#training loop
for epoch in range(epochs):

    total_epoch_loss = 0

    for batch_features, batch_labels in train_loader:

        #move data to gpu
        batch_features, batch_labels =batch_features.to(device), batch_labels.to(device)

        #forward pass
        outputs = model(batch_features)
        #loss calculation
        loss =criterion(outputs,batch_labels)
        #backpropagation
        optimizer.zero_grad()
        loss.backward()
        #update the parameters
        optimizer.step()

        total_epoch_loss = total_epoch_loss + loss.item()

    avg_loss = total_epoch_loss/len(train_loader)
    print(f"Epoch:{epoch +1} and Loss : {avg_loss}")


Epoch:1 and Loss : 0.6352872455318769
Epoch:2 and Loss : 0.4304986933817466
Epoch:3 and Loss : 0.38612620331346986
Epoch:4 and Loss : 0.3584607186516126
Epoch:5 and Loss : 0.3376494685560465
Epoch:6 and Loss : 0.3227646785800656
Epoch:7 and Loss : 0.30785389595727125
Epoch:8 and Loss : 0.29498188248028356
Epoch:9 and Loss : 0.2854692431092262
Epoch:10 and Loss : 0.27467057487616936
Epoch:11 and Loss : 0.26830568559716145
Epoch:12 and Loss : 0.25814215187107525
Epoch:13 and Loss : 0.24940819021190205
Epoch:14 and Loss : 0.24444738084822892
Epoch:15 and Loss : 0.23859191479658087
Epoch:16 and Loss : 0.23155898495018482
Epoch:17 and Loss : 0.22562562109405795
Epoch:18 and Loss : 0.220296459933122
Epoch:19 and Loss : 0.21206333881616593
Epoch:20 and Loss : 0.20960091057543953
Epoch:21 and Loss : 0.20624992236122489
Epoch:22 and Loss : 0.19986102367161462
Epoch:23 and Loss : 0.19530413204431535
Epoch:24 and Loss : 0.19312162966746838
Epoch:25 and Loss : 0.18764106344679993
Epoch:26 and Loss

In [ ]:
#set the model to eval mode
model.eval()

MyNN(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [ ]:
#evaluation
total = 0
correct = 0
with torch.no_grad():
    for batch_features,batch_labels in test_loader:

        #move data to gpu
        batch_features, batch_labels =batch_features.to(device), batch_labels.to(device)

        outputs = model(batch_features)
        _,predicted = torch.max(outputs,1)

        total = total + batch_features.shape[0]
        correct = correct + (predicted == batch_labels).sum().item()

    accuracy = correct/total
    print(accuracy)



0.8898333333333334
